### Imports and parameters

In [33]:
import copy as cp
import numpy as np
import numpy.random as rnd
import PIL.Image as img
import PIL.ImageDraw as imgdr
import PIL.ImageChops as chops
import scipy.spatial.distance as scidst

source_image = img.open("./source_img.png").convert('RGB')
generation_size = 5
max_generation_cycles = 500
mutation_chance = 0.25
mutation_speed = 10.0
selection_percent = 0.8

#source_image_arr = np.array(source_image).reshape(512*512*4)
#source_image_norm = np.sum(source_image_arr)

### Image Utils

In [36]:
def distance(image1,image2):
    w,h = image1.size
    #img1_arr, img2_arr = np.array(image1), np.array(image2)
    #image_res = img.new('RGBA',image1.size);
    #image_res = chops.subtract(image2,image1)
    img1_arr = np.array(image1).reshape(512*512*3)
    img2_arr = np.array(image2).reshape(512*512*3)
    #dot_val = np.dot(img1_arr,img2_arr)
    #norm1 = np.sum(img1_arr)
    #norm2 = np.sum(img2_arr)
    #res = dot_val#/(norm1*norm2)
    return scidst.cosine(img1_arr,img2_arr)

def get_random_color(transparency = 0):
    if transparency == 0:
        transparency = 255#np.random.randint(0,255)
    return (np.random.randint(255), np.random.randint(255),np.random.randint(255))#,transparency)

def change_color(color,dcomp):
    r,g,b = color
    r += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
    g += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
    b += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
    #a += int(np.clip(0,rnd.uniform(-dcomp,dcomp),255))
    return (r,g,b)

### Chromosome

In [19]:
class Gene:
    
    @staticmethod
    def generate_random():
        pass
    
    def draw_at(self,canvas):
        pass
    
    def mutate(self):
        pass
        
class CircleGene:
    
    def __init__(self,x,y,radius,color):
        self.x, self.y = x, y
        self.radius = radius
        self.color = color
        self.fitness = -1
        
    def draw_at(self,canvas):
        x, y = self.x, self.y
        r, c = self.radius, self.color
        canvas.ellipse((x-r,y-r,x+r,y+r),fill=c)
        
    def mutate(self):
        self.x += (rnd.rand()-0.5 ) * 2 * mutation_speed
        self.y += (rnd.rand()-0.5 ) * 2 * mutation_speed
        self.radius += (rnd.rand()-0.5) * 2 * mutation_speed
        self.color = change_color(self.color,rnd.rand()*10*mutation_speed)
        
    @staticmethod
    def generate_random(image):
        w,h = image.size
        x = np.random.uniform(0,w)
        y = np.random.uniform(0,h)
        r = np.random.uniform(0,16)
        #c = image.getpixel((x,y))
        gene = CircleGene(x,y,r,get_random_color(0))
        return gene
        
        
class Chromosome:
    
    current_image = None
    current_image_canvas = None
    
    def __init__(self,genes = [],genes_count = 600,size=(512,512):
        self.genes = genes
        self.fitness = -1
        self.genes_count = genes_count
        #self.image_size = size
        #self.image = img.new('RGBA',target_image.size)
        
    @staticmethod
    def generate_random(image,genes_count=80):
        genes = []
        for i in range(genes_count):
            gene = CircleGene.generate_random(image)
            genes.append(gene)
        chromosome = Chromosome(genes=genes,genes_count=genes_count)
        #chromosome.get_fitness(image)
        return chromosome
    
    def get_fitness(self,target_image):
        #if self.fitness != -1:
         #   return self.fitness
        w,h = target_image.size
        #self.image = img.new('RGBA',target_image.size)
        self.image = Chromosome.current_image.copy()
        canvas = imgdr.Draw(self.image, 'RGBA')
        for i in range(self.genes_count):
            self.genes[i].draw_at(canvas)
        return distance(target_image,self.image)
        #return self.fitness
    
    def mutate(self,):
        

    def __lt__(self, other):
        if isinstance(other,float):
            return self.fitness < other
        elif isinstance(other,Chromosome):
            return self.fitness < other.fitness
        else: 
            False
            
    def __eq__(self, other):
        return self.fitness == other.fitness
        
    def __hash__(self):
        return int(1000000000*self.fitness)
    
    def __str__(self):
        return "Chromosome :fitness = {}".format(self.fitness)



### Breeding

In [4]:
class Breeder:
    #inject self.engine
    
    def __call__(self,chromosomes):
        #return chromosomes in pairs
        pass
    
class InBreeder:
    
    def __init__(self):
        self.engine = None
        
    def __call__(self,chromosomes):
        res = []
        sorted_chs = chromosomes
        sorted_chs.sort(reverse=True)
        #sorted_chs = list(set(sorted_chs))
        size = int(len(sorted_chs))
        if size % 2 != 0:
            size = size - 1
        for i in range(0,size,2):
            res.append((sorted_chs[i],sorted_chs[i+1]))
        return res
    
class UsualBreeder:
    
    def __init__(self):
        self.engine = None
        
    def __call__(self,chromosomes):
        res = []
        sorted_chs = chromosomes
        size = len(sorted_chs)
        for i in range(0,int(size),2):
            res.append((sorted_chs[i],sorted_chs[i+1]))
        return res

### Mutation

In [5]:
class Mutator:
    #inject self.engine
    
    def __call__(self,chromosome):
        # mutation of chromosome
        pass

    
class SelectiveMutator:
    
    def __init__(self):
        self.engine = None
        
    def __call__(self,chromosome):
        chromosome.fitness = -1
        index = rnd.randint(0, chromosome.genes_count)
        chromosome.genes[index].mutate()
        chromosome.get_fitness(source_image)
        return chromosome


### Crossover

In [11]:
class Crossoverer:
    #inject self.engine
    
    def __call__(self,chromosome1,chromosome2):
        #return (new_chromosome1,new_chromosome2)
        pass
    
class SinglePointCrossoverer:
    
    def __init__(self):
        self.engine = None
    
    def __call__(self,chromosome1,chromosome2):
        if chromosome1.genes_count != chromosome2.genes_count:
            return (chromosome1,chromosome2)
        crossover_point = rnd.randint(chromosome1.genes_count)
        gc = chromosome1.genes_count
        new_ch1, new_ch2 = chromosome1, chromosome2
        for i in range(crossover_point):
            new_ch1.genes[i], new_ch2.genes[i] = chromosome2.genes[i], chromosome1.genes[i]
        new_ch1.fitness = -1
        new_ch2.fitness = -1
        return new_ch1, new_ch2
    
class RandomPointCrossoverer:
    
    #def __init__(self):
     #   self.engine = None
        
    def __call__(self,chromosome1,chromosome2):
        if chromosome1.genes_count != chromosome2.genes_count:
            return (chromosome1,chromosome2)
        crossover_point = rnd.randint(chromosome1.genes_count)
        for i in range(crossover_point):
            if rnd.rand() > 0.9:
                chromosome1.genes[i], chromosome2.genes[i] = chromosome2.genes[i], chromosome1.genes[i]
        return (chromosome1, chromosome2)

#ch = Chromosome.generate_random(source_image,genes_count=600)
#print(ch.get_fitness(source_image))
#ch.image

### Selection

In [7]:
class Selector:
    
    #inject self.engine
        
    def __call__(self,chromosome):
        #return true, if chromosome will survive
        #false overwise
        pass
    
class AverageSelector:
    
    def __init__(self):
        self.engine = None
        
    def __call__(self,chromosome):
        image = self.engine.target_image
        fit = chromosome.get_fitness(image)
        avg_fit = self.engine.average_fitness
        #print("average fit ",avg_fit," vs chromosome fit ",fit)
        return fit > avg_fit
    

### GA Engine

In [32]:
class Engine:
    
    def __init__(self,population_size=100):
        self.breeder = None
        self.mutator = None
        self.selector = None
        self.crossoverer = None
        self.target_image = None
        self.average_fitness = 0
        self.population_size = population_size
    
    def set_breeder(self,breeder):
        self.breeder = breeder
        breeder.engine = self
        
    def set_mutator(self, mutator):
        self.mutator = mutator
        mutator.engine = self
        
    def set_crossoverer(self, crossoverer):
        self.crossoverer = crossoverer
        crossoverer.engine = self
    
    def set_selector(self, selector):
        self.selector = selector
        selector.engine = self
        
    def finalize_setup(self):
        if self.breeder is None:
            self.set_breeder(InBreeder())
        if self.mutator is None:
            self.set_mutator(SelectiveMutator())
        if self.crossoverer is None:
            self.set_crossoverer(SinglePointCrossoverer())
        if self.selector is None:
            self.set_selector(AverageSelector())

    def run(self,target_image,required_fitness = 0.5):
        self.target_image = target_image
        self.finalize_setup()
        generation = []
        #init generation
        for i in range(self.population_size):
            generation.append(Chromosome.generate_random(target_image,genes_count=200))
        current_fitness = 0
        best_fit = 0
        iteration = 1
        Chromosome.current_image = img.new('RGB',source_image.size)
        #Chromosome.current_image_canvas = imgdr.Draw(Chromosome.current_image)
        while self.average_fitness < required_fitness:
            
            #compute fitness
            fitness_sum = 0
            fitness_best = 0
            for i in range(len(generation)):
                generation[i].fitness = -1
                fitness_chr = generation[i].get_fitness(target_image)
                if fitness_chr > fitness_best:
                    fitness_best = fitness_chr
                fitness_sum += fitness_chr
            self.average_fitness = float(fitness_sum) / self.population_size
            
            print("Generation average fitness: ",self.average_fitness, ", best fitness :",fitness_best)
            #for i in range(self.population_size):
            #    print(generation[i])
                
            #apply selector (change generation)
            prev_generation = cp.copy(generation)
            #generation = []
#             prev_generation.sort(reverse=True)
#             to_fill = self.population_size
#             for i in range(len(prev_generation)):
#                 chromosome = prev_generation[i]
#                 #if self.selector(chromosome):
#                 if i/self.population_size < 0.8:
#                     generation.append(chromosome)
#                     to_fill -= 1
#             #print("Survived  : ",self.population_size - to_fill)
            
#             half_at_max = len(prev_generation)
#             half_at_max = int(np.clip(0,to_fill*0.5,half_at_max))
#             for i in range(half_at_max):
#                 if to_fill <= 0:
#                     break
#                 generation.append(prev_generation[i])
#                 to_fill-=1
#             #print("Generated : ",to_fill)
#             for i in range(to_fill):
#                 generation.append(Chromosome.generate_random(target_image,genes_count=100))
            
            #print("pre breeding")
            #for i in range(self.population_size):
            #    print(generation[i])
            #make breeding
#             pairs = self.breeder(generation)
            
#             #print("post breeding")
#             #for i in range(len(pairs)):
#             #    a,b = pairs[i]
#             #    print(a)
#             #    print(b)
                
#             generation = []
#             for i in range(len(pairs)):
#                 ch1,ch2 = pairs[i]
#                 ch1new,ch2new = self.crossoverer(ch1,ch2)
#                 generation.append(ch1new)
#                 generation.append(ch2new)
                
            #mutations 
            #print(self.population_size,len(generation))
            for i in range(len(generation)):
                r = rnd.rand()
                if r >= mutation_chance: 
                     continue
                generation[i] = self.mutator(generation[i])
                
            fitness_sum = 0
            for i in range(len(generation)):
                generation[i].fitness = -1
                fitness_sum += generation[i].get_fitness(target_image)
            self.new_average_fitness = float(fitness_sum) / self.population_size
            
            if self.average_fitness > self.new_average_fitness:
                print("Generation degraded, reverting")
                generation = prev_generation
                continue
                
            for i in range(len(generation)):
                i_fit = generation[i].get_fitness(target_image)
                if i_fit > best_fit:
                    best_fit = generation[i]
                    Chromosome.current_image = best_fit.image
            if iteration % 10 == 0:
                best_fit.image.save("./out.png")
            iteration += 1
            
        return best_fit

### Main

In [37]:
def main():
    engine = Engine(10)
    #engine.set_breeder(UsualBreeder())
    engine.set_crossoverer(RandomPointCrossoverer())
    try:
        chromosome = engine.run(source_image,required_fitness=0.99)
        
        return chromosome.image
    except KeyboardInterrupt:
        print("interrupted")
    
main()

Generation average fitness:  0.553359503247275 , best fitness : 0.5937795592654258
Generation degraded, reverting
Generation average fitness:  0.5531976624931911 , best fitness : 0.5937795592654258
Generation average fitness:  0.044670850500789824 , best fitness : 0.05744364221944487
Generation average fitness:  0.044670850500789824 , best fitness : 0.05744364221944487
Generation average fitness:  0.04467994720006317 , best fitness : 0.05752824814320856
Generation degraded, reverting
Generation average fitness:  0.044635836270384546 , best fitness : 0.05752824814320856
Generation degraded, reverting
Generation average fitness:  0.04448349019432992 , best fitness : 0.0576094240642524
Generation average fitness:  0.04378708155877925 , best fitness : 0.057326687936981924
Generation degraded, reverting
Generation average fitness:  0.043525754724812994 , best fitness : 0.057326687936981924
Generation degraded, reverting
Generation average fitness:  0.04345218399075279 , best fitness : 0.057

Generation average fitness:  0.043986523135825704 , best fitness : 0.056706253056489575
Generation degraded, reverting
Generation average fitness:  0.04394271465432117 , best fitness : 0.05602261002230513
Generation degraded, reverting
Generation average fitness:  0.04387733519833562 , best fitness : 0.05602261002230513
Generation average fitness:  0.042256250159335706 , best fitness : 0.05401833949259727
Generation average fitness:  0.042256250159335706 , best fitness : 0.05401833949259727
Generation average fitness:  0.04209437981564986 , best fitness : 0.053526422999168544
Generation average fitness:  0.042149360143509576 , best fitness : 0.05359530882517227
Generation average fitness:  0.04206671536849632 , best fitness : 0.053434781230202844
Generation degraded, reverting
Generation average fitness:  0.04206262707483741 , best fitness : 0.053471988277795646
Generation average fitness:  0.04195385964356526 , best fitness : 0.053277143703498786
Generation degraded, reverting
Generat

Generation average fitness:  0.03768682385470268 , best fitness : 0.04527431817682381
Generation average fitness:  0.037396646718210955 , best fitness : 0.044295573860453574
Generation average fitness:  0.037726353418267036 , best fitness : 0.044624730782140576
Generation average fitness:  0.037439667715728986 , best fitness : 0.044249142680127806
Generation average fitness:  0.03784306185232306 , best fitness : 0.044702637837896186
Generation average fitness:  0.038315532374789754 , best fitness : 0.04506357267911887
Generation degraded, reverting
Generation average fitness:  0.03820377245229707 , best fitness : 0.04506357267911887
Generation average fitness:  0.03795822395172847 , best fitness : 0.04473152392194879
Generation degraded, reverting
Generation average fitness:  0.03763847172895657 , best fitness : 0.04473152392194879
Generation degraded, reverting
Generation average fitness:  0.03751772793104128 , best fitness : 0.04473152392194879
Generation degraded, reverting
Generati

Generation average fitness:  0.025611633050142357 , best fitness : 0.03457301854074213
Generation average fitness:  0.025854069895917032 , best fitness : 0.03485661540295881
Generation degraded, reverting
Generation average fitness:  0.025845942551459243 , best fitness : 0.03485661540295881
Generation degraded, reverting
Generation average fitness:  0.025841387739021136 , best fitness : 0.03485661540295881
Generation degraded, reverting
Generation average fitness:  0.02573318861307079 , best fitness : 0.03485661540295881
Generation average fitness:  0.02500976206792558 , best fitness : 0.03403565488951543
Generation average fitness:  0.02503250778859125 , best fitness : 0.0340570519433081
Generation degraded, reverting
Generation average fitness:  0.024780995665871674 , best fitness : 0.0340570519433081
Generation degraded, reverting
Generation average fitness:  0.024715583664013586 , best fitness : 0.0340570519433081
Generation average fitness:  0.024349772454010744 , best fitness : 0

Generation average fitness:  0.020131097175484503 , best fitness : 0.03241682389976719
Generation degraded, reverting
Generation average fitness:  0.020039218456814778 , best fitness : 0.0329020645263689
Generation degraded, reverting
Generation average fitness:  0.019945364679921463 , best fitness : 0.0329020645263689
Generation degraded, reverting
Generation average fitness:  0.01954613947186732 , best fitness : 0.0329020645263689
Generation average fitness:  0.017369793722086878 , best fitness : 0.0300701151394831
Generation degraded, reverting
Generation average fitness:  0.017327283636952018 , best fitness : 0.0300701151394831
Generation degraded, reverting
Generation average fitness:  0.017086530430157322 , best fitness : 0.0300701151394831
Generation average fitness:  0.015606991998997089 , best fitness : 0.028642508932748556
Generation degraded, reverting
Generation average fitness:  0.015497728501112718 , best fitness : 0.028642508932748556
Generation degraded, reverting
Gener

Generation degraded, reverting
Generation average fitness:  0.007692224258992219 , best fitness : 0.01750602882937269
Generation degraded, reverting
Generation average fitness:  0.007585266153701098 , best fitness : 0.01750602882937269
Generation degraded, reverting
Generation average fitness:  0.007317757993657003 , best fitness : 0.0178704109966481
Generation average fitness:  0.005621969514303526 , best fitness : 0.01663932183034167
Generation average fitness:  0.0053450559806330735 , best fitness : 0.016739187790462462
Generation degraded, reverting
Generation average fitness:  0.005123253395302308 , best fitness : 0.016739187790462462
Generation degraded, reverting
Generation average fitness:  0.004995089631955607 , best fitness : 0.016739187790462462
Generation average fitness:  0.004054152156167013 , best fitness : 0.015560925682744031
Generation average fitness:  0.005415215383225624 , best fitness : 0.017817638204263186
Generation average fitness:  0.005541667114051974 , best 